In [53]:
import duckdb
import polars as pl
import torch
from torch_geometric.data import Data
from torch_geometric.utils import is_undirected
from torch_sparse import coalesce
import numpy as np
from torch_geometric.utils import to_networkx
from utils import StoreDataset, LoadedDataset, create_pyg_graph_from_polars


In [46]:
con = duckdb.connect(database="../../data/02_primary/distanze.db")
CONTIGUI = '../../data/01_raw/contigui_data.parquet'

In [47]:
QUERY = """
    SELECT 
        trim(split_part(OR_DEST,' - ',1)) as OR, trim(split_part(OR_DEST,' - ',2)) as DEST, TEP_TOT, KM_TOT, TTP_TOT 
    FROM '../../data/01_raw/Italia/*/*.csv'
    """

In [48]:
df = pl.from_arrow(
    con.execute(
        QUERY
    ).fetch_arrow_table()
)
print(df.head())

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

shape: (5, 5)
┌───────┬───────┬─────────┬────────┬─────────┐
│ OR    ┆ DEST  ┆ TEP_TOT ┆ KM_TOT ┆ TTP_TOT │
│ ---   ┆ ---   ┆ ---     ┆ ---    ┆ ---     │
│ str   ┆ str   ┆ i64     ┆ f64    ┆ i64     │
╞═══════╪═══════╪═════════╪════════╪═════════╡
│ 66001 ┆ 66001 ┆ 0       ┆ 0.0    ┆ 0       │
│ 66001 ┆ 66055 ┆ 6       ┆ 4.9    ┆ 5       │
│ 66001 ┆ 66086 ┆ 9       ┆ 7.1    ┆ 7       │
│ 66001 ┆ 66031 ┆ 10      ┆ 8.1    ┆ 9       │
│ 66001 ┆ 66027 ┆ 13      ┆ 10.8   ┆ 11      │
└───────┴───────┴─────────┴────────┴─────────┘


In [49]:
df_cont = pl.read_parquet(CONTIGUI, columns = ['PRO_COM COMUNE','PRO_COM COMUNE ADIACENTE','source_file'])
df_cont = df_cont.with_columns(
    pl.col('source_file')\
      .str.extract(r"([^/]+)\.xlsx$", 1)  # capture the filename without .xlsx
      .str.slice(-4, 4)                   # grab the year
      .alias("year"),
      pl.col('PRO_COM COMUNE').alias('OR'),
      pl.col('PRO_COM COMUNE ADIACENTE').alias('DEST')
      )\
        .select(['OR', 'DEST', 'year'])

In [50]:
df_join = df_cont.join(df, on = ['OR', 'DEST'], how = 'left')

In [51]:
FILTER_YEAR = 2022
data = create_pyg_graph_from_polars(
    df_join.filter(pl.col('year')==str(FILTER_YEAR)), 
    "OR", "DEST", 
    ["TEP_TOT", "KM_TOT", "TTP_TOT"]
    )

In [58]:
if data.is_directed():
    edge_index, edge_attr = coalesce(
    data.edge_index, 
    data.edge_attr if hasattr(data, 'edge_attr') else None,
    m=data.num_nodes,
    n=data.num_nodes
    )
    data.edge_index = edge_index
    data.edge_attr  = edge_attr

In [59]:
# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Has isolated nodes: {data.has_isolated_nodes()}')
print(f'Has self-loops: {data.has_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

Number of nodes: 7890
Number of edges: 46446
Average node degree: 5.89
Has isolated nodes: False
Has self-loops: False
Is undirected: False


In [60]:
StoreDataset(
    data_list = [data],
    folder = "../../data/03_feature",
    filename = "italy_network"
)